In [172]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np

import scipy.signal as signal


# Kick Drum

In [178]:
sr = 44100
kick_dur = 0.5
freq0 = 150

kick_len = int(kick_dur*sr)

pitch_env = np.logspace(np.log10(1),np.log10(0.1), kick_len)
amp_env = np.logspace(np.log10(1),np.log10(0.01), kick_len)

freq = freq0*pitch_env

phase = 0
x_kick = np.zeros(kick_len)

for i in range(len(x_kick)):
    phase = phase + 2*np.pi*freq[i]/sr
    x_kick[i] = np.sin(phase)

x_kick = amp_env*x_kick

ipd.Audio(x_kick, rate=sr)



# Snare Drum

In [187]:
snare_dur = 0.2

snare_len = int(snare_dur*sr)

# bi-quad highpass filter
cut_off_freq = 500
Q = 0.5

theta = 2*np.pi*cut_off_freq/sr
alpha = np.sin(theta)/2/Q
b = [(1+np.cos(theta)), -2*(1+np.cos(theta)), (1+np.cos(theta))]
a = [(1+alpha), -2*np.cos(theta), (1-alpha)]

x_snare = (np.random.rand(snare_len)-0.5)/2
amp_env = np.logspace(np.log10(1),np.log10(0.01), snare_len)


x_snare = signal.lfilter(b,a,x_snare)

x_snare = amp_env*x_snare

ipd.Audio(x_snare, rate=sr)


# Closed Hi-Hat

In [188]:
snare_dur = 0.05

snare_len = int(snare_dur*sr)

# bi-quad highpass filter
cut_off_freq = 5000
Q = 0.5

theta = 2*np.pi*cut_off_freq/sr
alpha = np.sin(theta)/2/Q
b = [(1+np.cos(theta)), -2*(1+np.cos(theta)), (1+np.cos(theta))]
a = [(1+alpha), -2*np.cos(theta), (1-alpha)]

x_snare = (np.random.rand(snare_len)-0.5)/2
amp_env = np.logspace(np.log10(1),np.log10(0.01), snare_len)


x_snare = signal.lfilter(b,a,x_snare)

x_snare = amp_env*x_snare

ipd.Audio(x_snare, rate=sr)


# Define drum tones as a function

In [189]:
def gen_drum_type1(dur, freq0, pitch_decay_level, sr):
    
    sample_len = int(dur*sr)

    pitch_env = np.logspace(np.log10(1),np.log10(pitch_decay_level), sample_len)
    amp_env = np.logspace(np.log10(1),np.log10(0.01), sample_len)

    freq = freq0*pitch_env

    phase = 0
    output = np.zeros(sample_len)

    for i in range(sample_len):
        phase = phase + 2*np.pi*freq[i]/sr
        output[i] = np.sin(phase)

    output = amp_env*output

    return output
    
sr = 44100
x_kick = gen_drum_type1(0.5, 150, 0.01, sr)

ipd.Audio(x_kick, rate=sr)






In [190]:
def gen_drum_type2(dur, cut_off_freq, sr):
    
    sample_len = int(dur*sr)

    # bi-quad highpass filter
    #cut_off_freq = 500
    Q = 0.5

    theta = 2*np.pi*cut_off_freq/sr
    alpha = np.sin(theta)/2/Q
    b = [(1+np.cos(theta)), -2*(1+np.cos(theta)), (1+np.cos(theta))]
    a = [(1+alpha), -2*np.cos(theta), (1-alpha)]

    output = (np.random.rand(sample_len)-0.5)/2
    amp_env = np.logspace(np.log10(1),np.log10(0.01), sample_len)


    output = signal.lfilter(b, a, output)

    output = amp_env*output

    return output


sr = 44100
x_snare = gen_drum_type2(0.2, 500, sr)

ipd.Audio(x_snare, rate=sr)
    

In [112]:
x_close_hihat = gen_drum_type2(0.05, 2000, sr)

ipd.Audio(x_close_hihat, rate=sr)


# Rhythm Generation 

In [192]:
beat_h = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
beat_s = [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1]
beat_k = [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1]
      
unit_time = 0.25 # sec

cur = 0 

kick_len = len(x_kick)
snare_len = len(x_snare)
hihat_len = len(x_close_hihat)

x_beat = np.zeros(int(unit_time*sr*(len(beat_h)-1))+kick_len)


for i in range(len(beat_h)):

    if beat_h[i] == 1:
        x_beat[cur:cur+hihat_len] = x_beat[cur:cur+hihat_len] + x_close_hihat
        
    if beat_s[i] == 1:
        x_beat[cur:cur+snare_len] = x_beat[cur:cur+snare_len] + x_snare
        
    if beat_k[i] == 1:
        x_beat[cur:cur+kick_len] = x_beat[cur:cur+kick_len] + x_kick
    
    cur = cur + int(unit_time*sr) 

ipd.Audio(x_beat, rate=sr)
